In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from sklearn.svm import SVC

# -------------------------------
# CONFIGURATION
# -------------------------------
DATASET_DIR = "alzheimers-gm-mask1"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
TARGET_COUNT = 1600
CLASS_NAMES = ["AD", "CN", "MCI"]
NUM_WORKERS = 4

# -------------------------------
# DATA AUGMENTATION
# -------------------------------
def augment_data(dataset_dir, class_names, target_count, img_size):
    print("Starting dataset augmentation...")
    augment_datagen = ImageDataGenerator(
        rotation_range=7,
        zoom_range=0.1,
        fill_mode='nearest'
    )
    for class_name in class_names:
        class_dir = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        augment_count = target_count - len(files)
        if augment_count > 0:
            aug_generator = augment_datagen.flow_from_directory(
                dataset_dir,
                classes=[class_name],
                target_size=img_size,
                batch_size=1,
                save_to_dir=class_dir,
                save_prefix='augmented_',
                save_format='png'
            )
            for _ in range(augment_count):
                next(aug_generator)

augment_data(DATASET_DIR, CLASS_NAMES, TARGET_COUNT, IMG_SIZE)

# -------------------------------
# DATASET LOADING
# -------------------------------
img_size = 224
common_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])
dataset = datasets.ImageFolder(root=DATASET_DIR, transform=common_transform)
dataset_size = len(dataset)
train_size = int(0.6 * dataset_size)
val_size = int(0.2 * dataset_size)
test_size = dataset_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Print number of images in each split
print(f"Number of training images: {train_size}")
print(f"Number of validation images: {val_size}")
print(f"Number of test images: {test_size}")

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define multi-scale CNN model
class MultiScaleEfficientNet(nn.Module):
    def __init__(self):
        super(MultiScaleEfficientNet, self).__init__()
        self.backbone = EfficientNet.from_pretrained('efficientnet-b0')
        self.backbone._fc = nn.Identity()

        self.branch1 = nn.Sequential(
            nn.Conv2d(1280, 256, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.5)
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(1280, 256, kernel_size=5, padding=2),
            nn.LeakyReLU(),
            nn.MaxPool2d(3),
            nn.Dropout(0.5)
        )
        self.branch3 = nn.Sequential(
            nn.Conv2d(1280, 256, kernel_size=7, padding=3),
            nn.LeakyReLU(),
            nn.MaxPool2d(4),
            nn.Dropout(0.5)
        )

        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(256 * 3, len(CLASS_NAMES))

    def forward(self, x, extract_features=False):
        features = self.backbone.extract_features(x)
        b1 = self.adaptive_pool(self.branch1(features))
        b2 = self.adaptive_pool(self.branch2(features))
        b3 = self.adaptive_pool(self.branch3(features))
        concat = torch.cat([b1, b2, b3], dim=1)
        flattened = concat.view(concat.size(0), -1)

        if extract_features:
            return flattened
        else:
            return self.classifier(flattened)

# Initialize model
model = MultiScaleEfficientNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
EPOCHS = 4

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

    train_acc = total_correct / total_samples * 100
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}, Accuracy: {train_acc:.4f}%")

# Feature extraction function
def extract_features(dataloader, model):
    model.eval()
    features_list, labels_list = [], []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Extracting Features"):
            images = images.to(device)
            feats = model(images, extract_features=True)
            features_list.append(feats.cpu().numpy())
            labels_list.append(labels.numpy())

    return np.vstack(features_list), np.hstack(labels_list)

# Extract features for all sets
X_train, y_train = extract_features(train_loader, model)
X_val, y_val     = extract_features(val_loader, model)
X_test, y_test   = extract_features(test_loader, model)

from sklearn.model_selection import GridSearchCV

# Hyperparameter tuning for SVM
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__gamma': ['scale', 'auto'],
    'svc__kernel': ['rbf', 'linear']
}

# Create SVM pipeline
svm = make_pipeline(StandardScaler(), SVC(probability=True))

# Perform Grid Search to find the best hyperparameters
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best SVM parameters from Grid Search
best_svm = grid_search.best_estimator_

# Evaluate best SVM on the training set
y_train_pred = best_svm.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)

# Evaluate best SVM on the validation set
y_val_pred = best_svm.predict(X_val)
val_acc = accuracy_score(y_val, y_val_pred)

# Evaluate best SVM on the test set
y_test_pred = best_svm.predict(X_test)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"✅ SVM Train Accuracy: {train_acc * 100:.4f}%")
print(f"✅ SVM Validation Accuracy: {val_acc * 100:.4f}%")
print(f"✅ SVM Test Accuracy: {test_acc * 100:.4f}%")
print(f"Best Hyperparameters: {grid_search.best_params_}")

# Show classification reports
print("\nTraining Classification Report:")
print(classification_report(y_train, y_train_pred, target_names=CLASS_NAMES, digits=4))

print("\nValidation Classification Report:")
print(classification_report(y_val, y_val_pred, target_names=CLASS_NAMES, digits=4))

print("\nTest Classification Report:")
print(classification_report(y_test, y_test_pred, target_names=CLASS_NAMES, digits=4))

# Confusion Matrix for the test set
print("Confusion Matrix for Test Set:")
print(confusion_matrix(y_test, y_test_pred))



In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
import numpy as np

# Assuming:
# - all_labels: true labels from test set
# - hybrid_preds: predictions from hybrid SVM model
# - baseline_preds: predictions from baseline model

# Replace with your actual predictions
hybrid_preds = y_test_pred  # from hybrid model
baseline_preds = all_preds  # from baseline model

# Build 2x2 contingency table
b = 0  # Hybrid correct, Baseline wrong
c = 0  # Hybrid wrong, Baseline correct

for true, h_pred, b_pred in zip(y_test, hybrid_preds, baseline_preds):
    if h_pred == true and b_pred != true:
        b += 1
    elif h_pred != true and b_pred == true:
        c += 1

# Contingency table format:
#        | Baseline correct | Baseline wrong |
# --------------------------------------------
# Hybrid correct |    a         |       b      |
# Hybrid wrong   |    c         |       d      |

table = [[0, b],
         [c, 0]]

result = mcnemar(table, exact=False, correction=True)

print(f"McNemar's test statistic = {result.statistic}")
print(f"p-value = {result.pvalue}")

if result.pvalue < 0.05:
    print("✅ Statistically significant difference between models (p < 0.05)")
else:
    print("❌ No statistically significant difference between models (p ≥ 0.05)")


McNemar's test statistic = 374.2142857142857
p-value = 2.2627017630882975e-83
✅ Statistically significant difference between models (p < 0.05)
